In [1]:
from pandas import read_csv , concat , DataFrame
from sklearn.preprocessing import LabelEncoder , StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error , mean_squared_error
encoder = LabelEncoder()
scaler = StandardScaler()

In [2]:
file = read_csv("D:\\datasets\\honey_purity_dataset\\honey_purity_dataset.csv")
df = file.copy()
df.sample(10)

,CS,Density,WC,pH,EC,F,G,Pollen_analysis,Viscosity,Purity,Price
47498,2.69,1.66,23.46,5.93,0.87,45.19,29.32,Sage,6598.87,0.66,612.96
157018,5.11,1.71,21.04,7.25,0.79,49.92,24.45,Rosemary,2860.68,0.95,714.93
157778,5.94,1.29,12.08,6.49,0.75,34.35,30.17,Orange Blossom,2754.77,0.99,249.37
135800,9.42,1.70,21.74,6.59,0.71,32.54,27.41,Eucalyptus,2050.75,0.66,454.72
135045,8.35,1.74,23.40,2.96,0.86,34.69,25.60,Thyme,8662.45,0.68,624.62
87899,4.35,1.57,18.63,3.96,0.76,34.12,24.50,Wildflower,5433.74,0.82,262.90
127705,9.03,1.54,20.40,6.64,0.85,46.33,22.16,Sage,7966.22,0.64,589.79
48398,1.90,1.64,15.36,3.09,0.87,36.33,24.44,Acacia,8474.85,0.82,443.87
34231,8.98,1.39,16.66,3.38,0.88,32.55,34.67,Wildflower,8944.40,1.00,321.85
164094,7.56,1.72,14.65,2.56,0.88,32.05,31.45,Thyme,5825.55,0.68,624.62


In [3]:
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247903 entries, 0 to 247902
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   CS               247903 non-null  float64
 1   Density          247903 non-null  float64
 2   WC               247903 non-null  float64
 3   pH               247903 non-null  float64
 4   EC               247903 non-null  float64
 5   F                247903 non-null  float64
 6   G                247903 non-null  float64
 7   Pollen_analysis  247903 non-null  object 
 8   Viscosity        247903 non-null  float64
 9   Purity           247903 non-null  float64
 10  Price            247903 non-null  float64
dtypes: float64(10), object(1)
memory usage: 20.8+ MB


In [4]:
df.describe()

,CS,Density,WC,pH,EC,F,G,Viscosity,Purity,Price
count,247903.000000,247903.000000,247903.000000,247903.000000,247903.000000,247903.000000,247903.000000,247903.000000,247903.000000,247903.000000
mean,5.500259,1.535523,18.502625,4.996047,0.799974,34.970573,32.501006,5752.893888,0.824471,594.807644
std,2.593947,0.187824,3.748635,1.444060,0.057911,8.655898,7.226290,2455.739903,0.139417,233.627972
min,1.000000,1.210000,12.000000,2.500000,0.700000,20.000000,20.000000,1500.050000,0.610000,128.720000
25%,3.260000,1.370000,15.260000,3.750000,0.750000,27.460000,26.230000,3627.880000,0.660000,433.000000
50%,5.500000,1.540000,18.510000,4.990000,0.800000,34.970000,32.490000,5753.770000,0.820000,612.960000
75%,7.740000,1.700000,21.750000,6.250000,0.850000,42.470000,38.760000,7886.650000,0.970000,770.220000
max,10.000000,1.860000,25.000000,7.500000,0.900000,50.000000,45.000000,9999.970000,1.000000,976.690000


In [5]:
df.Pollen_analysis.unique()

array(['Blueberry', 'Alfalfa', 'Chestnut', 'Borage', 'Sunflower',
       'Orange Blossom', 'Acacia', 'Tupelo', 'Clover', 'Wildflower',
       'Thyme', 'Sage', 'Avocado', 'Lavender', 'Eucalyptus', 'Buckwheat',
       'Rosemary', 'Heather', 'Manuka'], dtype=object)

In [6]:
Pollen_analysis = df.Pollen_analysis
df = df.drop(['Pollen_analysis'],axis=1)

In [7]:
Corr = df.corr()
print(Corr['Purity'].sort_values(ascending=False))
print('------------------------------------')
print(Corr['Price'].sort_values(ascending=False))

Purity       1.000000
Price        0.432581
CS           0.079770
Viscosity    0.012572
EC           0.000151
G            0.000059
F           -0.001149
WC          -0.028894
Density     -0.108834
pH          -0.230855
Name: Purity, dtype: float64
------------------------------------
Price        1.000000
Purity       0.432581
CS           0.035166
Viscosity    0.009632
EC          -0.000571
F           -0.002041
G           -0.004815
WC          -0.014381
Density     -0.050518
pH          -0.100714
Name: Price, dtype: float64


In [8]:
lbl_pur = df.Purity
lbl_price = df.Price
df = df.drop(['Purity','Price'],axis=1)
encoded_Pollen_analysis = DataFrame(encoder.fit_transform(Pollen_analysis),columns=['Pollen_analysis'])
final = concat([df,encoded_Pollen_analysis],axis=1)
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247903 entries, 0 to 247902
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   CS               247903 non-null  float64
 1   Density          247903 non-null  float64
 2   WC               247903 non-null  float64
 3   pH               247903 non-null  float64
 4   EC               247903 non-null  float64
 5   F                247903 non-null  float64
 6   G                247903 non-null  float64
 7   Viscosity        247903 non-null  float64
 8   Pollen_analysis  247903 non-null  int32  
dtypes: float64(8), int32(1)
memory usage: 16.1 MB


In [9]:
scaled_df = DataFrame(scaler.fit_transform(final),columns=final.columns)

In [10]:
xtr , xts , ytr , yts = train_test_split(scaled_df,lbl_pur,test_size=0.2,random_state=32)

In [11]:
model = GradientBoostingRegressor()

In [12]:
model.fit(xtr,ytr)

GradientBoostingRegressor()

model's performance Purity column

In [13]:
preds = model.predict(xtr)
mae = mean_absolute_error(ytr,preds)
mse = mean_squared_error(ytr,preds)
print(f'mae : {mae}\nmse :{mse}')

mae : 0.020055186535096782
mse :0.0008855111750323575


In [14]:
preds = model.predict(xts)
mae = mean_absolute_error(yts,preds)
mse = mean_squared_error(yts,preds)
print(f'mae : {mae}\nmse :{mse}')

mae : 0.02004810824197721
mse :0.0008884496798345658


Model's performance on Price column

In [15]:
xtr , xts , ytr , yts = train_test_split(scaled_df,lbl_price,test_size=0.2,random_state=32)

In [16]:
model.fit(xtr,ytr)

GradientBoostingRegressor()

In [17]:
preds = model.predict(xtr)
mae = mean_absolute_error(ytr,preds)
mse = mean_squared_error(ytr,preds)
print(f'mae : {mae}\nmse :{mse}')

mae : 39.66225848332605
mse :2625.6027505893408


In [18]:
preds = model.predict(xts)
mae = mean_absolute_error(yts,preds)
mse = mean_squared_error(yts,preds)
print(f'mae : {mae}\nmse :{mse}')

mae : 39.678231586242596
mse :2628.638298242441
